## Image reconstruction using Maximum Likeliood Expectation Maximization (MLEM)


Maximum Likeliood Expectation Maximization (MLEM) is one of the most used iterative image reconstruction algorithms used to reconstruct PET and SPECT images. Detailed background information on this algorithm for PET and SPECT image reconstruction is available in [this video](https://www.youtube.com/watch?v=CHKOSYdf47c) and in [this video](https://www.youtube.com/watch?v=Z70n5NCw9BY). Moreover, background information on the concept of maximum likelihood approaches are available [here](https://www.youtube.com/watch?v=uTa7g_h4c1E). Background information on PET and SPECT is available [here](https://www.youtube.com/watch?v=M8DOzE2d0dw) and [here](https://www.youtube.com/watch?v=4mrtq8CeLvo&list=PLKkWkQgtnBS1tWAE3-TL1-MDKY9EUJTFP&index=2).

### Learning objective

The aim of this notebook is to implement the iterative MLEM for reconstruction of simulated 2D PET data and study the influence of the number of used MLEM updates on the quality of the reconstructed image.

### The MLEM update algorithm

Since MLEM is an iterative algorithm, it tells us how to calculated an updated image $x^{n+1}$ based on a current image $x^n$ and measured PET emission data $y$ via

\begin{equation}
x^{n+1} = \frac{x^n}{P^T \mathbb{1}} P^T \left( \frac{y}{\bar{y}(x^n)} \right) \ ,
\label{eq:MLEM_update}
\tag{1}
\end{equation}

where

\begin{equation}
\bar{y}(x^n) = Px^n + s \ .
\label{eq:fwd_model}
\tag{2}
\end{equation}

Eq. (2) is the so-called forward model that calculates which data $\bar{y}$ to expect based on the current image $x^n$. The linear operator $P$ includes a model of the PET acquisition physics (e.g. calculation of line integrals, and correction for attenuation and detector sensitivities). Moreover, $s$ denotes the contribution of random and scattered coincidences. In many books, the $P$ is called the *forward model* and the application of $P$ to $x^n$ is called the *(corrected) forward projection* of $x^n$ which maps from image space into the data (sinogram) space.
Accordingly, the adjoint (transposed) operator $P^T$ - often also called the back projection - maps from data (sinogram) space into image space. Note that (i) $\mathbb{1}$ is a data set (sinogram) full of ones and (ii) the divisions in (1) are to be understood point-wise. 

**In this notebook, we assume that $P$, $P^T$, $s$ are known and we will provide simplistic 2D implementations of them.**

### Module import section

In [ ]:
# import of modules that we need in this notebook
# make sure that utils.py is placed in the same directory 

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from utils import RotationBased2DProjector, PETAcquisitionModel, test_images

# needed to get inline matplotlib plots in an IPython notebook
%matplotlib inline

### Input parameter section

In [ ]:
# input parameter for our demo

# number of pixels for our images to be simulated and reconstructed
npix = 100
# pixel size in mm
pix_size_mm = 4

# set the default color map to Greys (white to black) for showing images
plt.rcParams['image.cmap'] = 'Greys'
# set the random seed
np.random.seed(1)

### Setup of ground truth images

In [ ]:
# generate the ground truth activity (emission) and attenuation images that we use for the data simulation
em_img, att_img = test_images(npix)

In [ ]:
# show the true activity (emission) image and the attenuation image
fig, ax = plt.subplots(1,2)
im0 = ax[0].imshow(em_img)
im1 = ax[1].imshow(att_img)
fig.colorbar(im0, ax = ax[0], location = 'bottom')
fig.colorbar(im1, ax = ax[1], location = 'bottom')
ax[0].set_title('ground truth activity image')
ax[1].set_title('ground truth attenuation image')
fig.tight_layout()

print(f'image shape {em_img.shape}')

Note that the attenuation image is needed in the forward model to model the effect of photon attenuation.

### Data simulation 

In [ ]:
# setup the forward projector"
# the acq_model object is an abstract representation of the linear operator P (and also it's adjoint)
proj = RotationBased2DProjector(npix, pix_size_mm = pix_size_mm, num_subsets = 1)
acq_model = PETAcquisitionModel(proj, att_img)

In [ ]:
# generate noise free data by applying the acquisition model to our simulated emission image
noise_free_data = acq_model.forward(em_img)

In [ ]:
# add poisson noise to the data
noisy_data = np.random.poisson(noise_free_data)

In [ ]:
# show the noise-free and noisy simulated emission data (sinogram)
fig2, ax2 = plt.subplots(1,2)
im02 = ax2[0].imshow(noise_free_data)
im12 = ax2[1].imshow(noisy_data)
fig2.colorbar(im02, ax = ax2[0], location = 'bottom')
fig2.colorbar(im12, ax = ax2[1], location = 'bottom')
ax2[0].set_xlabel('radial element')
ax2[1].set_xlabel('radial element')
ax2[0].set_ylabel('view')
ax2[1].set_ylabel('view')
ax2[0].set_title('noise-free data')
ax2[1].set_title('noisy data (to be reconstructed)')
fig2.tight_layout()

print(f'data (sinogram) shape {noise_free_data.shape}')

### Image reconstruction

To reconstruct the simualted data sinogram $y$ with the MLEM algorithm as given in Eq. (1), we need to know how to evaluate the forward model in Eq. (2) and how to evaluate the adjoin operator $P^T$ (the backprojection).

For a given image $x$, the forward step $\bar{y}(x) = Px + s$ can be calculated via: 

In [ ]:
# setup a random image x
x    = np.zeros((npix,npix))
x[30:35, 35:40] = 1
ybar = acq_model.forward(x)  

Note that in this notebook, the value of the contamination $s$ is fixed and included in the call to ```acq_model.forward()```.

Show the image $x$ and the expected data $\bar{y}(x)$.

In [ ]:
fig3, ax3 = plt.subplots(1,2)
im03 = ax3[0].imshow(x)
im13 = ax3[1].imshow(ybar)
fig3.colorbar(im03, ax = ax3[0], location = 'bottom')
fig3.colorbar(im13, ax = ax3[1], location = 'bottom')
ax3[1].set_xlabel('radial element')
ax3[1].set_ylabel('view')
ax3[0].set_title(r'image $x$')
ax3[1].set_title(r'expected data $\bar{y}(x) = Px + s$')
fig3.tight_layout()

print(f'shape of x {x.shape}')
print(f'shape of ybar {ybar.shape}')

To evaluate $P^T r$ (the adjoint operator $P^T$ applied to a given data set / sinogram $r$) you can call:

In [ ]:
r = ybar.copy()
z = acq_model.adjoint(r)

Show the sinogram $r$ and the image $z$

In [ ]:
fig4, ax4 = plt.subplots(1,2)
im04 = ax4[0].imshow(r)
im14 = ax4[1].imshow(z)
fig4.colorbar(im04, ax = ax4[0], location = 'bottom')
fig4.colorbar(im14, ax = ax4[1], location = 'bottom')
ax4[0].set_xlabel('radial element')
ax4[0].set_ylabel('view')
ax4[0].set_title(r'sinogram $r$')
ax4[1].set_title(r'image $z = P^T r$')
fig4.tight_layout()

print(f'shape of r {r.shape}')
print(f'shape of z {z.shape}')

### Now it is your turn

1. With the knowledge from the cells above, implent an iterative MLEM reconstruction to reconstruct the data stored in ```noisy_data``` sinogram in the cell below.
2. Run 200 updates (iterations) to reconstruct the data and display the reconstructed image.
3. How does the number of used updated (iterations) influence the reconstructed image? Visualize reconstructions after 5, 25, 100, 200 and updates (iterations) by showing the reconstructed images and also a line profile through the center.

Note:
- before running the first update, you have to initialize the image $x^0$. Use ```acq_model.init_image()``` to get an image that contains 1s in the field-of-view that can be reconstructed and 0s in the background.
- only reconstruct the image in pixels where the initial image is not 0 (which corresponds to the pixels that can be reconstructed because they are within the simulated PET scanner ring.)

In [ ]:
# initialize our reconstruction here
x_0 = acq_model.init_image()


#------------------------------------------------------
#------------------------------------------------------
#------------------------------------------------------

# MLEM updates to be implemented here

n_updates = 200

# pixel indices that can be reconstructed
recon_inds = np.where(x_0 > 0)

# allocate empty array for recon after every update
recons = np.zeros((n_updates,) + x.shape)

# calculate the backprojection of a sinogram full of ones (the sensitivity image)
sens_img = acq_model.adjoint(np.ones(noisy_data.shape))

x = x_0.copy()

for i in range(n_updates):
  print(f'iteration {(i+1):04}', end = '\r')
  # calculate the forwward step (expectation of data given image x)
  ybar  = acq_model.forward(x)
  # calcualte the ratio of measure data and expected data
  ratio =  noisy_data / ybar
  # update the image
  x[recon_inds] *= acq_model.adjoint(ratio)[recon_inds] 
  x[recon_inds] /= sens_img[recon_inds]

  recons[i,...] = x


#------------------------------------------------------
#------------------------------------------------------
#------------------------------------------------------

In [ ]:
# visualize reconstruction
# show initial image
fig5, ax5 = plt.subplots(1,5, figsize = (15,3))
im50 = ax5[0].imshow(x_0, vmin = 0, vmax = 1.2*em_img.max())
fig5.colorbar(im50, ax = ax5[0], location = 'bottom')
ax5[0].set_title(r'initial image $x^0$')

# show MLEM reconstruction here
# replace x_0 with reconstruction(s) here

for i,update in enumerate([5,25,100,200]):
  im = ax5[i+1].imshow(recons[update-1,...], vmin = 0, vmax = 1.2*em_img.max())
  fig5.colorbar(im, ax = ax5[i+1], location = 'bottom')
  ax5[i+1].set_title(f'reconstr. {update} MLEM updates')

fig5.tight_layout()

In [ ]:
# visualize a line profile through the reconstructions
fig6, ax6 = plt.subplots(figsize = (9,6))

for i,update in enumerate([5,25,100,200]):
  ax6.plot(recons[update-1,npix//2,:], label = f'{update} updates')  

ax6.legend()
ax6.grid(ls = ':')
fig6.tight_layout()